In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from os import walk, getcwd
#import h5py

##All npy files are stored in the train_data folder:
mypath = "./train_data/"
txt_name_list = []
for (dirpath, dirnames, filenames) in walk(mypath):
    if filenames != '.DS_Store':       ##Ugh mac junk
        txt_name_list.extend(filenames)
        break


In [3]:
len(txt_name_list)

50

In [4]:
x_train = []
x_test = []
y_train = []
y_test = []
xtotal = []
ytotal = []
slice_train = int(80000/len(txt_name_list))  ###Setting value to be 80000 for the final dataset
i = 0
seed = np.random.randint(1, 10e6)

In [5]:
##Creates test/train split with quickdraw data
for txt_name in txt_name_list:
    txt_path = mypath + txt_name
    x = np.load(txt_path)
    x = x.astype('float32') / 255.    ##scale images
    y = [i] * len(x)  
    np.random.seed(seed)
    np.random.shuffle(x)
    np.random.seed(seed)
    np.random.shuffle(y)
    x = x[:slice_train]
    y = y[:slice_train]
    if i != 0: 
        xtotal = np.concatenate((x,xtotal), axis=0)
        ytotal = np.concatenate((y,ytotal), axis=0)
    else:
        xtotal = x
        ytotal = y
    i += 1
x_train, x_test, y_train, y_test = train_test_split(xtotal, ytotal, test_size=0.2, random_state=42)

In [6]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np

Using TensorFlow backend.


In [7]:
batch_size = 128
num_classes = 50
epochs = 25

# input image dimensions
img_rows, img_cols = 28, 28

x_train = x_train.reshape(64000,28,28,1)
x_test = x_test.reshape(16000,28,28,1)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (64000, 28, 28, 1)
64000 train samples
16000 test samples


In [8]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Use tf.cast instead.
Train on 64000 samples, validate on 16000 samples
Epoch 1/25
64000/64000 [==============================] - 301s 5ms/step - loss: 2.5427 - acc: 0.3524 - val_loss: 1.6206 - val_acc: 0.5951
Epoch 2/25
64000/64000 [==============================] - 311s 5ms/step - loss: 1.7895 - acc: 0.5361 - val_loss: 1.4055 - val_acc: 0.6411
Epoch 3/25
64000/64000 [==============================] - 294s 5ms/step - loss: 1.5845 - acc: 0.5837 - val_loss: 1.2882 - val_acc: 0.6706
Epoch 4/25
64000/64000 [==============================] - 297s 5ms/step - loss: 1.4539 - acc: 0.6177 - val_loss: 1.1899 - val_acc: 0.6904
Epoch 5/25
64000/64000 [==============================] - 320s 5ms/step - loss: 1.3552 - acc: 0.6417 - val_loss: 1.1431 - val_acc: 0.7031
Epoch 6/25
64000/64000 [==============================] - 228s 4ms/step - loss: 1.2742 - acc: 0.6631 - val_loss: 1.1286 - val_acc: 0.7047
Epoch 7/25
64000/64000 [==============================] - 214s 3ms/step - 

In [21]:
model.save('quickdraw_50.h5')